In [3]:
import numpy as np
import random
import time

class Particle:
    swarm_best_position = None
    swarm_best_value = float('inf')

    def __init__(self, T, m, execution_times, c_inertia, c_social, c_cognitive):
        self.T = T
        self.m = m
        self.execution_times = execution_times
        self.c_inertia = c_inertia
        self.c_social = c_social
        self.c_cognitive = c_cognitive

        self.position = np.random.randint(0, m, size=T)
        self.velocity = np.random.uniform(-1, 1, size=T)
        self.value = self.calculate_completion_time()

        self.personal_best_position = self.position.copy()
        self.personal_best_value = self.value

        if Particle.swarm_best_position is None or self.value < Particle.swarm_best_value:
            Particle.swarm_best_position = self.position.copy()
            Particle.swarm_best_value = self.value

    def calculate_completion_time(self):
        processor_times = [0] * self.m
        for task, processor in enumerate(self.position):
            processor_times[processor] += self.execution_times[task][processor]
        return max(processor_times)

    def update_velocity(self):
        r_s = np.random.random(len(self.position))
        r_c = np.random.random(len(self.position))
        social_velocity = self.c_social * r_s * (Particle.swarm_best_position - self.position)
        cognitive_velocity = self.c_cognitive * r_c * (self.personal_best_position - self.position)
        self.velocity = self.c_inertia * self.velocity + social_velocity + cognitive_velocity

    def move(self):
        self.update_velocity()
        self.position = np.clip(self.position + self.velocity, 0, self.m - 1).astype(int)
        self.value = self.calculate_completion_time()

        if self.value < self.personal_best_value:
            self.personal_best_position = self.position.copy()
            self.personal_best_value = self.value
            if self.value < Particle.swarm_best_value:
                Particle.swarm_best_position = self.position.copy()
                Particle.swarm_best_value = self.value

In [4]:
def pso(T, m, execution_times, num_iters, swarm_size, c_inertia, c_social, c_cognitive):
    Particle.swarm_best_position = None
    Particle.swarm_best_value = float('inf')
    
    swarm = [Particle(T, m, execution_times, c_inertia, c_social, c_cognitive) for _ in range(swarm_size)]
    for _ in range(num_iters):
        for p in swarm:
            p.move()
    
    return Particle.swarm_best_position, Particle.swarm_best_value

# Testiranje različitih ulaza
test_cases = [
    (4, 2),
    (6, 3),
    (8, 4),
    (10, 5),
    (15, 6),
]

results = []

print("PSO testovi...\n")
for T, m in test_cases:
    execution_times = [[random.randint(1, 10) for _ in range(m)] for _ in range(T)]
    print(f"Testiram za T = {T}, m = {m}...")
    
    start_time = time.time()
    best_schedule, best_time = pso(T, m, execution_times, num_iters=100, swarm_size=30, c_inertia=0.7, c_social=1.5, c_cognitive=1.5)
    execution_duration = time.time() - start_time
    
    results.append((T, m, execution_duration, best_time))
    print(f"  -> Završeno za {execution_duration:.2f} sekundi. Najbolje pronađeno vreme: {best_time}\n")

print("Rezultati testova:\n")
for T, m, duration, best_time in results:
    print(f"Za T = {T}, m = {m}: Završeno za {duration:.2f} sekundi. Najbolje vreme: {best_time}")


PSO testovi...

Testiram za T = 4, m = 2...
  -> Završeno za 0.08 sekundi. Najbolje pronađeno vreme: 6

Testiram za T = 6, m = 3...
  -> Završeno za 0.05 sekundi. Najbolje pronađeno vreme: 10

Testiram za T = 8, m = 4...
  -> Završeno za 0.05 sekundi. Najbolje pronađeno vreme: 7

Testiram za T = 10, m = 5...
  -> Završeno za 0.06 sekundi. Najbolje pronađeno vreme: 6

Testiram za T = 15, m = 6...
  -> Završeno za 0.04 sekundi. Najbolje pronađeno vreme: 9

Rezultati testova:

Za T = 4, m = 2: Završeno za 0.08 sekundi. Najbolje vreme: 6
Za T = 6, m = 3: Završeno za 0.05 sekundi. Najbolje vreme: 10
Za T = 8, m = 4: Završeno za 0.05 sekundi. Najbolje vreme: 7
Za T = 10, m = 5: Završeno za 0.06 sekundi. Najbolje vreme: 6
Za T = 15, m = 6: Završeno za 0.04 sekundi. Najbolje vreme: 9
